<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/BlackUP_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Main Code

In [ ]:
%cd drive/MyDrive/DeepLearning/BlackUp/Dataset

studyData = pd.read_csv("KOSPI200_20y_dataset.csv");
testData = pd.read_csv("005930_10y_dataset.csv")

closePrice = studyData["Close"].values
openPrice = studyData["Open"].values
highPrice = studyData["High"].values
lowPrice = studyData["Low"].values
tradingVolume = studyData["Trading"].values
chageRate = studyData["ChageRate"].values